### Important Chain methods

In [3]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

In [13]:
prompt = PromptTemplate.from_template(
    """You are a helpful assistant returning output in just plain text. Give me a short bio of the {celebrity} in 1 line"""
)

chain = prompt | openai_llm | StrOutputParser()

##### ```.invoke```

In [6]:
res = chain.invoke({"celebrity":"Trisha krishnan"})

print(res)

Trisha Krishnan is a prominent Indian actress and model, primarily known for her work in Tamil and Telugu films.


##### ```.batch```

In [8]:
celebrities = [{"celebrity":"Trisha Krisnnan"}, {"celebrity":"Ajith Kumar"}, {"celebrity":"Samantha Ruth Prabhu"}]

responses = chain.batch(celebrities)

for res in responses:
    print(res, "\n")

Trisha Krishnan is a prominent Indian actress and model, primarily known for her work in Tamil and Telugu films. 

Ajith Kumar is a prominent Indian actor, primarily working in Tamil cinema, known for his action-oriented roles and racing career. 

Samantha Ruth Prabhu is a prominent Indian actress primarily known for her work in Telugu and Tamil cinema. 



##### ```.stream```

In [9]:
celebrities = [{"celebrity":"Trisha Krisnnan"}, {"celebrity":"Ajith Kumar"}, {"celebrity":"Samantha Ruth Prabhu"}]

for celeb in celebrities:
    chunks = chain.stream(celeb)
    for chunk in chunks:
        print(chunk)

Tr
isha Krishnan
 is a prominent Indian actress and model, primarily known for her work in Tamil and
 Telugu films.

Aj
ith Kumar is
 a prominent Indian actor, primarily working in Tamil cinema, known for his action-
oriented roles and racing career.

Samantha
 Ruth Prabhu is
 a prominent Indian actress primarily known for her work in Telugu and Tamil cinema.



##### ```.ainvoke```

In [10]:
# how .invoke performs currently

p1 = chain.invoke({"celebrity": "Thalapathy Vijay"})
p2 = chain.invoke({"celebrity": "Trisha Krishnan"})
p3 = chain.invoke({"celebrity": "Samantha Ruth Prabhu"})

print(p1, p2, p3)


Thalapathy Vijay is a highly popular Indian actor and philanthropist, primarily working in Tamil cinema. Trisha Krishnan is a prominent Indian actress and model, primarily known for her work in Tamil and Telugu films. Samantha Ruth Prabhu is a prominent Indian actress primarily known for her work in Telugu and Tamil cinema.


In [14]:
# how .ainvoke performs with .await

async def case2():

    r1 = await chain.ainvoke({"celebrity": "Thalapathy Vijay"})
    r2 = await chain.ainvoke({"celebrity": "Trisha Krishnan"})
    r3 = await chain.ainvoke({"celebrity": "Samantha Ruth Prabhu"})
    
    print(r1, "\n", r2, "\n", r3)

await case2()


Thalapathy Vijay is a renowned Indian film actor and playback singer, predominantly known for his work in Tamil cinema. 
 Trisha Krishnan is an award-winning Indian film actress and model, primarily known for her work in South Indian cinema. 
 Samantha Ruth Prabhu is an Indian actress and model who has established a successful career in the Telugu and Tamil film industries.


In [16]:
# Making it truly parallel processing using python's asyncio

import asyncio

async def case3():

    t1 = asyncio.create_task(chain.ainvoke({"celebrity": "Thalapathy Vijay"}))
    t2 = asyncio.create_task(chain.ainvoke({"celebrity": "Trisha Krishnan"}))
    t3 = asyncio.create_task(chain.ainvoke({"celebrity": "Samantha Ruth Prabhu"}))

    results = await asyncio.gather(t1, t2, t3)

    # results = await asyncio.gather(
    #     chain.ainvoke({"celebrity": "Thalapathy Vijay"}),
    #     chain.ainvoke({"celebrity": "Trisha Krishnan"}),
    #     chain.ainvoke({"celebrity": "Samantha Ruth Prabhu"})
    # )
    
    for result in results:
        print(result,"\n")

await case3()


Thalapathy Vijay is a renowned Indian film actor and playback singer who predominantly works in Tamil cinema. 

Trisha Krishnan is an award-winning Indian film actress and model, primarily known for her work in South Indian cinema. 

Samantha Ruth Prabhu is an Indian actress and model who has established a successful career in the Telugu and Tamil film industries. 



##### ```.abatch```

In [17]:
# Using .abatch instead of python's asyncio for simplicity

inputs = [
    {"celebrity": "Thalapathy Vijay"},
    {"celebrity": "Trisha Krishnan"},
    {"celebrity": "Samantha Ruth Prabhu"}
]

results = await chain.abatch(inputs)

for result in results:
    print(result,"\n")

Thalapathy Vijay is a renowned Indian film actor and playback singer, predominantly known for his work in the Tamil cinema industry. 

Trisha Krishnan is an award-winning Indian film actress and model, primarily known for her work in South Indian cinema. 

Samantha Ruth Prabhu is an Indian actress and model who has established a successful career in the Telugu and Tamil film industries. 



##### ```.astream```

In [18]:
# Using .astream for asyn streaming instead of blocking .stream

celebrities = [{"celebrity":"Trisha Krisnnan"}, {"celebrity":"Ajith Kumar"}, {"celebrity":"Samantha Ruth Prabhu"}]


for celeb in celebrities:
    chunks = chain.astream(celeb)
    async for chunk in chunks:
        print(chunk)


Tr
isha
 Krish
nan
 is
 an
 award
-winning
 Indian
 film
 actress
 and
 model
,
 primarily
 known
 for
 her
 work
 in
 South
 Indian
 cinema
.


Aj
ith
 Kumar
 is
 a
 prominent
 Indian
 film
 actor
 best
 known
 for
 his
 work
 in
 Tamil
 cinema
.


S
aman
tha
 Ruth
 Pr
ab
hu
 is
 an
 Indian
 actress
 and
 model
 who
 has
 established
 a
 successful
 career
 in
 the
 Tel
ugu
 and
 Tamil
 film
 industries
.



##### ```.astream_events```

In [ ]:
# Using .astream_events for asyn streaming instead of blocking .stream

celebrities = [{"celebrity":"Trisha Krisnnan"}, {"celebrity":"Ajith Kumar"}, {"celebrity":"Samantha Ruth Prabhu"}]

for celeb in celebrities:
    chunks = chain.astream_events(celeb)
    async for chunk in chunks:
        print(chunk)

##### Demonstrating difference between ```.stream``` and ```.astream``` with timer

In [ ]:
import asyncio

celebrities = [{"celebrity":"Trisha Krisnnan"}, {"celebrity":"Ajith Kumar"}, {"celebrity":"Samantha Ruth Prabhu"}]

async def background_timer():
    for i in range(5):
        await asyncio.sleep(1)
        print(f"[Timer] {i+1} seconds passed")

async def run_streams():
    timer_task = asyncio.create_task(background_timer())
    
    for celeb in celebrities:
        chunks = chain.astream(celeb)
        async for chunk in chunks:
            print(chunk, end="")
        print("\n--- Done ---")
    
    await timer_task

await run_streams()

Trisha Krishnan is an award-winning Indian film actress and model, primarily known for her work in South Indian cinema.
--- Done ---
Ajith Kumar is a prominent Indian film actor best known for his work in Tamil cinema.
--- Done ---
Samantha Ruth Prabhu is an Indian actress and model who has established a successful career in the Telugu and Tamil film industries.
--- Done ---
[Timer] 1 seconds passed
[Timer] 2 seconds passed
[Timer] 3 seconds passed
[Timer] 4 seconds passed
[Timer] 5 seconds passed
